## CS:GO Player Form Prediction

To accompany our article/blog, this notebook is designed to show how we can use the simple variables such as the number of kills and deaths a player has in a given game in order to predict how they "should" perform on their next game. We first do a bunch of preprocessing and data manipulation. Then we use a recurrent neural network (specifically LSTM) to predict our measure of player form. Our real model of player form incorporates many more variables that we have extracted from the demo files of professional games, which give greater insight into player form than kills/deaths alone.

But anyway here is an open and fun way to look at using data to predict the outcome of professional CSGO matches

In [1]:
# Python imports
import pandas as pd
import numpy as np
import csv
import theano as th
from theano import tensor as T


WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


*Le data imports* 

Clearly you need to have the correct "matchdata" csv file in the same directory as this notebook...

In [2]:
dat = pd.read_csv('matchdata.csv',encoding = "ISO-8859-1")

C:\Users\Maximilian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (82,83,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Variable Names...

Here we can see a nice list of all the variables that our data gives us

In [3]:
list(dat.columns.values)

['Day',
 'Match ID',
 'Map',
 'Team 1',
 'Team 2',
 'Winning Team',
 'Score Team 1 as T',
 'Score Team 2 as CT',
 'Score Team 1 as CT',
 'Score Team 2 as T',
 'Player 1 Name',
 'Player 1 UserName',
 'Player 1 Kills',
 'Player 1 Deaths',
 'Player 1 KDA',
 'Player 1 ADR',
 'Player 2 Name',
 'Player 2 UserName',
 'Player 2 Kills',
 'Player 2 Deaths',
 'Player 2 KDA',
 'Player 2 ADR',
 'Player 3 Name',
 'Player 3 UserName',
 'Player 3 Kills',
 'Player 3 Deaths',
 'Player 3 KDA',
 'Player 3 ADR',
 'Player 4 Name',
 'Player 4 UserName',
 'Player 4 Kills',
 'Player 4 Deaths',
 'Player 4 KDA',
 'Player 4 ADR',
 'Player 5 Name',
 'Player 5 UserName',
 'Player 5 Kills',
 'Player 5 Deaths',
 'Player 5 KDA',
 'Player 5 ADR',
 'Player 6 Name',
 'Player 6 UserName',
 'Player 6 Kills',
 'Player 6 Deaths',
 'Player 6 KDA',
 'Player 6 ADR',
 'Player 7 Name',
 'Player 7 UserName',
 'Player 7 Kills',
 'Player 7 Deaths',
 'Player 7 KDA',
 'Player 7 ADR',
 'Player 8 Name',
 'Player 8 UserName',
 'Player 8 

In [4]:
# Need a function that pulls out only a certain teams data:

def team_data(team): # probably by index...
    df = dat.loc[(dat.ix[:,3] == team) |  (dat.ix[:,4] == team) ]
    return(df)

# make a player data one...

def player_data(player): # can define by indx later as well...
    df = dat.loc[(dat.ix[:,11] == player) |  (dat.ix[:,17] == player)|  (dat.ix[:,23] == player)|  (dat.ix[:,29] == player)|  (dat.ix[:,35] == player)|  (dat.ix[:,41] == player) |  (dat.ix[:,47] == player) |  (dat.ix[:,53] == player) |  (dat.ix[:,59] == player) |  (dat.ix[:,65] == player)]
    return(df)

# make function that says which team a player is on (1 or 2)...
def player_side(player, match): # match is the match # as integer

    dato = player_data(player).reset_index(drop=True) 
    N = len(dato)
    i = match
    
    side = []
    # loop over each row...
    if( (dato.ix[i,11] == player) |  (dato.ix[i,17] == player)|  (dato.ix[i,23] == player)|  (dato.ix[i,29] == player)|  (dato.ix[i,35] == player) ):
        side = 1
    elif( (dato.ix[i,41] == player) |  (dato.ix[i,47] == player) |  (dato.ix[i,53] == player) |  (dato.ix[i,59] == player) |  (dato.ix[i,65] == player) ):
        side = 2
    else:
        side = 0
    return(side)


def team_score(player, match):
    '''
    Returns team scores, enemy scores and team deaths (TD)...
    '''
    
    side = player_side(player, match)
    dato = player_data(player).reset_index(drop=True) 
    i = match
    if(side == 1):
        teamscore = dato.ix[match,6] +dato.ix[match,8]
        enemyscore = dato.ix[match,7] + dato.ix[match,9]
        TK = (dato.ix[i,12]) + (dato.ix[i,18])+  (dato.ix[i,24])+(dato.ix[i,30])+ (dato.ix[i,36])
        TD = (dato.ix[i,13] == player) + (dato.ix[i,19])+  (dato.ix[i,25])+(dato.ix[i,31])+ (dato.ix[i,37])
        
    elif(side ==2 ):
        teamscore = dato.ix[match,7] +dato.ix[match,9]
        enemyscore = dato.ix[match,6] +dato.ix[match,8]
        TK = dato.ix[i,42] + (dato.ix[i,48])+  (dato.ix[i,54])+(dato.ix[i,60] )+ (dato.ix[i,66])
        TD = dato.ix[i,43]+ (dato.ix[i,19])+  (dato.ix[i,55])+(dato.ix[i,61] )+ (dato.ix[i,67])
        
    else:
        teamscore = "ERROR"
        enemyscore = "UNLUGGY UCE"
        
        
    return( teamscore, enemyscore, TK, TD)



In [6]:
TS, ES, tk, td = team_score('GeT_RiGhT', 10)

print('team score is:', TS )
print('enemy score is:', ES )
print('team kills are:', tk )
print('team deaths are:', td)


team score is: 16
enemy score is: 7
team kills are: 89.0
team deaths are: 76.0


In [18]:
'''
get_metrics returns the players kills, KDR, teamscore, enemyscore, team kills and enemy kills at specified time...
'''
def get_metrics(player, match):
    TS, ES, tk, td = team_score(player, match)
    # get the new player 
    dato = player_data(player).reset_index(drop=True) 
    i = match
    if( dato.ix[i,11] == player ):
        kills = dato.ix[i,12]
        deaths = dato.ix[i,13]
        kd = dato.ix[i,12]/dato.ix[i,13]    
    elif( dato.ix[i,17] == player ):
        kills = dato.ix[i,18]
        deaths = dato.ix[i,19]
        kd = dato.ix[i,18]/dato.ix[i,19]
    elif( dato.ix[i,23] == player ):
        kills = dato.ix[i,24]    
        deaths = dato.ix[i,25]
        kd = dato.ix[i,24]/dato.ix[i,25]
    elif( dato.ix[i,29] == player ):
        kills = dato.ix[i,30]  
        deaths = dato.ix[i,31]
        kd = dato.ix[i,30]/dato.ix[i,31]
    elif( dato.ix[i,35] == player ):
        kills = dato.ix[i,36]
        deaths = dato.ix[i,37]
        kd = dato.ix[i,36]/dato.ix[i,37]
    elif( dato.ix[i,41] == player ):
        kills = dato.ix[i,42]    
        deaths = dato.ix[i,43]
        kd = dato.ix[i,42]/dato.ix[i,43]
    elif( dato.ix[i,47] == player ):
        kills = dato.ix[i,48]       
        deaths = dato.ix[i,49]
        kd = dato.ix[i,48]/dato.ix[i,49]
    elif( dato.ix[i,53] == player ):
        kills = dato.ix[i,54]      
        deaths = dato.ix[i,55]
        kd = dato.ix[i,54]/dato.ix[i,55]
    elif( dato.ix[i,59] == player ):
        kills = dato.ix[i,60] 
        deaths = dato.ix[i,61]
        kd = dato.ix[i,60]/dato.ix[i,61]
    elif( dato.ix[i,65] == player ):
        kills = dato.ix[i,66] 
        deaths = dato.ix[i,67]
        kd = dato.ix[i,66]/dato.ix[i,67]        
     
    k_TK = kills / tk
    kDR_A = kd / (np.log(1+TS)*(TS - ES+1))
    p_p = np.log(((kills*kills)/(deaths)+1))+np.log(((k_TK)*(kDR_A))/(deaths/td)+1)
    
    return(kills, deaths, kd, TS, ES, tk ,td, k_TK, kDR_A, p_p)            
                
'''
Now make function that makes CSV/df for the player and their past performance...
'''   

def make_inputs(player): # uses the player and gets all inputs from this...
    datta = player_data(player).reset_index(drop=True) 
    N = np.shape(datta)[0] # length of the data frame...
    
    # initialize the vectors...
    kill = []
    kdr = []
    scorediff = []
    kill_cont = [] # contribution of kills
    death_cont = [] # contriubution of deaths
    kdr_adjust = []
    player_perf = []
    
    print(N)
    
    # loop over each row to get data:
    for row in range(N):
        k,d,KDR, TS,ES,tk,td,k_TK,kDR_A, p_p = get_metrics('GeT_RiGhT', row)
        kill.append(k)
        kdr.append(KDR)
        scorediff.append(TS-ES)
        kill_cont.append(k_TK)
        death_cont.append(d/td)
        kdr_adjust.append(kDR_A)
        player_perf.append(p_p)
        
    # add all as columns to the dataframe...    
    newdat = pd.DataFrame([kill,kdr,scorediff,kill_cont, death_cont, kdr_adjust, player_perf])
    
    # but data is a tensor - so turn into tensor
    
    return(newdat)

In [20]:
#frame = make_inputs('fnx') # save as "frame" to use in function later - although will really call this from inside the function for 
# get_tensor
#get_metrics('GeT_RiGhT',3)
make_inputs('Sico')

40


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,21.000000,15.000000,20.000000,19.000000,23.000000,23.000000,25.000000,30.000000,17.000000,14.000000,...,19.000000,21.000000,16.000000,27.000000,24.000000,27.000000,20.000000,25.000000,22.000000,15.000000
1,1.166667,0.833333,1.111111,1.187500,1.352941,0.958333,3.571429,3.333333,0.850000,0.823529,...,0.950000,1.312500,1.333333,1.500000,1.600000,1.421053,2.222222,1.923077,0.956522,0.833333
2,-4.000000,4.000000,-7.000000,5.000000,5.000000,0.000000,11.000000,9.000000,-5.000000,-9.000000,...,6.000000,5.000000,11.000000,3.000000,8.000000,4.000000,13.000000,10.000000,0.000000,-6.000000
3,0.238636,0.159574,0.303030,0.182692,0.232323,0.186992,0.280899,0.312500,0.202381,0.202899,...,0.215909,0.203883,0.179775,0.270000,0.250000,0.245455,0.229885,0.277778,0.191304,0.185185
4,0.193548,0.246575,0.225000,0.186047,0.195402,0.258065,0.159091,0.121622,0.246914,0.193182,...,0.259740,0.242424,0.260870,0.191489,0.283019,0.215909,0.272727,0.185714,0.237113,0.191489
5,-0.151617,0.058826,-0.080425,0.069856,0.079588,0.345646,0.105046,0.117652,-0.085516,-0.049504,...,0.047901,0.077209,0.039217,0.132359,0.062748,0.100314,0.056025,0.061706,0.344992,-0.069505
6,3.031733,2.640053,3.030466,3.226003,3.559819,3.360810,4.673123,4.879253,2.664939,2.474684,...,2.986113,3.415008,3.132748,3.896807,3.727711,3.780958,3.862633,3.981670,3.338580,2.533107


## TENSORRRRRRRRRRZZZZZZZZZZZZZ

*Le Tensor* construction. Imagine a matrix, now imagine a stack of matrices for each input sample - we also make sure to zero-pad the input matrices, this way we have conformable arrays when implementing the recurrent net


In [52]:
def get_tensor(dataframe, K = 7, T = "default"):  # standardize the T window size, and K variables...
    '''
    T is window size
    K is number of input variables...
    N is number of input samples...
    '''
    N = np.shape(dataframe)[1]
    if(T == "default"):
        T = N # standardize T window
    # otherwise we have already defined T by function...
        
    tens = np.zeros((N,T,K)) # iniitalizes tensor shape... (From a given 'N', and known 'T' and 'K')

    '''
    basically need to generate a new dataframe for each 'N'...
    N is given by the number of time steps as well - as T
    ...
    each dataframe is a TxK array of zeros (using np.zeros)   
    '''

    # loop over each N:

    for i in range(N):

        # generate dataframe of zeros
        df = np.zeros((T,K)) # this is of size T by K...
    
        # add for the first one...
        df[0] = dataframe.ix[:K-1,N-1] # since frame is a pandas dataframe... (from make_inputs command...)
    
        for j in range(1,N-i):
            df[j] = dataframe.ix[:K-1,N-i-j] # since frame is a pandas dataframe... (from make_inputs command...)
        
        tens[i] = df # adds to the tensor...
    '''
    The result of this will be fed into get_targs... (Just FYI)
    '''
    #return(np.shape(tens)) # return the shape of the tensor - change to the actual tensor if working correct..
    return(tens)  # add this instead of above line if want actual tensor output...
    # it works!! (well the shape says so) so we will return the actual object instead ^^
               
        
def form_calc(kill,kdr,scorediff,kill_cont, death_cont, kdr_adjust):
    '''
    This calculates the form of a player given the input variables...
    '''
    # score = np.log((K**2)/D) + np.log( ( (K/TK)*(K/D)/ np.log(1+TS)*(TS-ES) )/(D/TD) ) # - old version...
    # Need to extrapolate team score: - since the winner *usually* gets 16 this gives us
    if(scorediff < 0):
        TS = 16 + scorediff
    else:
        TS = scorediff
        
    score = np.log(kill*kdr) + np.log( ((kill_cont*kdr)/(np.log(1+TS)*scorediff))/death_cont)  # new version
    # except we are currently 
    
    return(score)
    
    # Don't need this anymore...
    
    
def get_targs(tensor): 
    '''
    This function takes the tensor, removes the final entry,
    uses that to calculate the "target" then returns a tensor of
    one less "N" size than orignally. It also returns a vector of
    targets.
    '''
    # get N, T and K dimensions:
    N = np.shape(tensor)[0]
    T = np.shape(tensor)[1]
    K = np.shape(tensor)[2]-1  # target is given as the final dimension - so remove...
    
    # remove last record... ()
    newtens = np.delete(tensor,N - 1,0)
    
    # for each record, split tensor into input and target components... - so "T" will drop one dimension...
    #targz = np.zeros((N-1,1)) ## this is a vector
    targz = []
    inputs = np.zeros((N-1,T-1,K)) # this is still a 3D tensor, but dimensions have changed slightly
    
    for n in range(N-1): # looping over each sample...
        '''
        inputs are: kill,kdr,scorediff,kill_cont, death_cont, kdr_adjust , output is performance
        '''
        
        targz.append(tensor[n,1,6]/np.mean(tensor[n,1:T-n,6]))
        
        inputs[n-1] = tensor[n-1,1:T,:K]
        
        # can comment out the below if required
        print('inputs at',n,'are:',tensor[n,1,0],tensor[n,1,1],tensor[n,1,2],tensor[n,1,3],tensor[n,1,4],tensor[n,1,5])
        # that should do the trick
        
    return(targz, inputs)

In [46]:
TENZ = get_tensor(make_inputs('Sico')) # using the data for "sico"

40


In [35]:
np.shape(TENZ)

(40, 40, 7)

In [53]:
tz , iz =  get_targs(TENZ)

inputs at 0 are: 15.0 0.833333333333 -6.0 0.185185185185 0.191489361702 -0.0695053985707
inputs at 1 are: 22.0 0.95652173913 0.0 0.191304347826 0.237113402062 0.344992292386
inputs at 2 are: 25.0 1.92307692308 10.0 0.277777777778 0.185714285714 0.0617056160603
inputs at 3 are: 20.0 2.22222222222 13.0 0.229885057471 0.272727272727 0.0560247815658
inputs at 4 are: 27.0 1.42105263158 4.0 0.245454545455 0.215909090909 0.10031384573
inputs at 5 are: 24.0 1.6 8.0 0.25 0.283018867925 0.0627477553537
inputs at 6 are: 27.0 1.5 3.0 0.27 0.191489361702 0.132358546449
inputs at 7 are: 16.0 1.33333333333 11.0 0.179775280899 0.260869565217 0.0392173470961
inputs at 8 are: 21.0 1.3125 5.0 0.203883495146 0.242424242424 0.0772091520954
inputs at 9 are: 19.0 0.95 6.0 0.215909090909 0.25974025974 0.0479011882388
inputs at 10 are: 22.0 1.69230769231 11.0 0.247191011236 0.309523809524 0.049775863622
inputs at 11 are: 20.0 1.66666666667 4.0 0.21978021978 0.24 0.117652041288
inputs at 12 are: 24.0 1.2 -10.0 

In [54]:
tz

[0.78796325962839253,
 1.0327559654859835,
 1.2327804758568104,
 1.2037085139067123,
 1.1851539636466146,
 1.1748616088560364,
 1.2346978832777604,
 0.99994063937359312,
 1.0900330547343995,
 0.95600347526260399,
 1.1772051007059785,
 1.1704887046264756,
 1.080523537163715,
 0.4108524021878886,
 1.1912978731377126,
 1.3180196276737399,
 1.1541486101477423,
 0.86895211574679021,
 1.0772709826001614,
 0.51017749647967259,
 1.3626438979964635,
 1.0497005731972477,
 0.92112458902049665,
 0.96969965027008542,
 0.81754673748760176,
 1.2088981409262278,
 0.81425101010171563,
 0.49877104501287339,
 0.99503674678224718,
 0.69667231213716696,
 0.73001570640075386,
 0.76047496819278027,
 1.3462912180288045,
 1.3683926385876741,
 1.0623942487454898,
 1.1431346946494005,
 1.0878417922198504,
 1.0688495841404315,
 1.0]

In [55]:
iz

array([[[ 15.        ,   0.83333333,  -6.        ,   0.18518519,
           0.19148936,  -0.0695054 ],
        [ 22.        ,   0.95652174,   0.        ,   0.19130435,
           0.2371134 ,   0.34499229],
        [ 25.        ,   1.92307692,  10.        ,   0.27777778,
           0.18571429,   0.06170562],
        ..., 
        [ 19.        ,   1.1875    ,   5.        ,   0.18269231,
           0.18604651,   0.0698559 ],
        [ 20.        ,   1.11111111,  -7.        ,   0.3030303 ,
           0.225     ,  -0.0804249 ],
        [ 15.        ,   0.83333333,   4.        ,   0.15957447,
           0.24657534,   0.05882602]],

       [[ 22.        ,   0.95652174,   0.        ,   0.19130435,
           0.2371134 ,   0.34499229],
        [ 25.        ,   1.92307692,  10.        ,   0.27777778,
           0.18571429,   0.06170562],
        [ 20.        ,   2.22222222,  13.        ,   0.22988506,
           0.27272727,   0.05602478],
        ..., 
        [ 20.        ,   1.11111111,  -7.  

#### But this is a problem as in practice we are getting division by 0! - needs to be fixed...

## LSTM Time!

Now that we have our input data and targets we can use these to split into test and training data set, then run them through an LSTM neural network

In [57]:
def test_train(inputs, targs, percent = 30):
    '''
    Take input and target data then split them by the given percentage ratio
    "percent" must be between 1 and 50 - signalling no more training data splitting than testing 
    '''
    N = np.shape(inputs[0])
    N_train = ((100-percent)/100)*N
    # train set...
    train_i = inputs[:N_train,:,:]
    train_t = targs[:N_train]
    # test set...
    test_i =  inputs[N_train+1:,:,:]
    test_t = targs[N_train+1:]
    
    return(train_i,train_t,test_i,test_t)
    

### Define the LSTM Architecture

In [ ]:
in_neurons = np.shape(X_train)[2]   
out_neurons = 1 # since regression output  
hidden_neurons = 6

model = Sequential()  
model.add(LSTM(output_dim=hidden_neurons, input_dim=in_neurons, return_sequences=False)) 
model.add(Dense(output_dim=out_neurons, input_dim=hidden_neurons))
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer= 'rmsprop' )  


### Train the Model

In [ ]:
#(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data
model.fit(X_train, y_train, batch_size=1, nb_epoch=1, validation_split=0.05)  

predicted = model.predict(X_test)  
rmse = np.sqrt(((predicted - y_test) ** 2).mean(axis=0))


### Plotting stuff

In [ ]:
fig, ax = plt.subplots()

ax.plot(pd.DataFrame(y_test))
ax.plot(pd.DataFrame(predicted))


## API Demonstration

Below shows some examples of uses from this API

In [17]:
# test form calculation with different scenarios...
# this demonstrates the use of form_calc...
'''
good - easy win
'''
p1 = form_calc( K=22,D=4,TK = 23+22+13+12+13, TD = 12+10+7+11+4,TS=16,ES = 1 )
'''
average... (slight win) Coldzera (SK) vs fnatic...
'''
p2 = form_calc( K=16,D=13,TK = 23+19+20+16+15, TD = 17+13+18+11+13,TS=16,ES = 9 )
'''
good... (slight) win) fnx (SK) vs fnatic...
'''
p3 = form_calc( K=23,D=13,TK = 23+19+20+16+15, TD = 17+13+18+11+13,TS=16,ES = 9 )

print(p1)
print(p2)
print(p3)

9.23727443252
4.04407206672
5.49569404148


#### Below it might be useful to use a lookup table of players...

This can be done by the **show_players()** command...
Which returns a frame of each player.

I haven't actually made this command yet - but is on its way!

In [ ]:
# see the fellas we can use...
show_players()

### Team_score API call

This is used as player name followed by their last nth match. i.e. team_score("Get_RiGhT",0) shows the team information from get_right's last game... 

In [34]:
TS, ES, tk, td = team_score('GeT_RiGhT', 3)

# then we can print these variables out:

print('team score is:', TS )
print('enemy score is:', ES )
print('team kills are:', tk )
print('team deaths are:', td)


team score is: 16
enemy score is: 11
team kills are: 104.0
team deaths are: 86.0


**Below needs to be turned into a function - currently just testing that the data goes into the 3 dimensional array correctly...**

In [59]:
# manually defining N T and K although they will be decided by the data we are using from "make_inputs"
N = np.shape(frame)[1]
T = N # full sized time window...
K = 6
#
tens = np.zeros((N,T,K)) # iniitalizes tensor shape... (From a given 'N', and known 'T' and 'K')
#tens[0] = np.random.randn(T,K) # this is the dataframe at first sample...

'''
basically need to generate a new dataframe for each 'N'...
N is given by the number of time steps as well - as T
...
each dataframe is a TxK array of zeros (using np.zeros)

'''

# loop over each N:
#i =  # for the 5th one...
for i in range(N):
#if True == True: # comment this out for testing...
    # generate dataframe of zeros
    df = np.zeros((T,K)) # this is of size T by K...
    
    # add for the first one...
    df[0] = frame.ix[:,N-1] # since frame is a pandas dataframe... (from make_inputs command...)
    
    for j in range(1,N-i):
        df[j] = frame.ix[:,N-i-j] # since frame is a pandas dataframe... (from make_inputs command...)
        
    tens[i] = df # adds to the tensor...


The following cells visualise how the new tensor appears and how the the original data frame looks - there are still a few patches that need fixing...

In [8]:
print(tens[1])

NameError: name 'tens' is not defined

In [57]:
print(frame)

         0          1          2          3          4          5    \
0  21.000000  15.000000  20.000000  19.000000  23.000000  23.000000   
1   1.166667   0.833333   1.111111   1.187500   1.352941   0.958333   
2  -4.000000   4.000000  -7.000000   5.000000   5.000000   0.000000   
3   0.238636   0.159574   0.303030   0.182692   0.232323   0.186992   
4   0.193548   0.246575   0.225000   0.186047   0.195402   0.258065   
5  -0.151617   0.058826  -0.080425   0.069856   0.079588   0.345646   

         6          7          8          9      ...            267       268  \
0  25.000000  30.000000  17.000000  14.000000    ...      10.000000  6.000000   
1   3.571429   3.333333   0.850000   0.823529    ...       0.526316  0.272727   
2  11.000000   9.000000  -5.000000  -9.000000    ...     -11.000000 -9.000000   
3   0.280899   0.312500   0.202381   0.202899    ...       0.172414  0.093750   
4   0.159091   0.121622   0.246914   0.193182    ...       0.256757  0.229167   
5   0.105046   0

In [203]:
#n = 1 # number of samples...
#df = np.array(df)
#KK = np.reshape(df,(n,np.shape(df)[1],np.shape(df)[0]) )
#np.shape(KK)
J = [[[[1,2],[1,2]]],[[[1,2]]]]
J[0]
print(np.shape(J))
#get_tensor(df,2)
print(J[0])
#print(KK[0][0])

(2, 1)
[[[1, 2], [1, 2]]]


In [138]:
#np.shape((player_data('GeT_RiGhT').reset_index(drop=True) ))[0]
df = make_inputs('GeT_RiGhT')
df.to_csv('getright data.csv') # save this to csv

537


In [74]:
# win ratio metric:

def win_ratio(team):
    # get team data...
    dato = team_data(team).reset_index(drop=True)
    #print('The subset data length is:', len(dato))
    # then iterate over every row:
    WL = []
    #print(np.shape(data)[0])
    for row in range(len(dato)):
        # print(dat.ix[row,5])    # this here jsut for testing...
        if (dato.ix[row,5]) == team : # this is if they are the winning team - name is in the winning team col...
            WL.append(1)
            
        else:
            WL.append(0)
            
    #return(WL)
    return(np.sum(WL)/len(WL), np.std(WL)) # this is the Win ratio metric... and variance (STD)
    

In [77]:
win_ratio('EnVyUs') # returns mean and stdev

(0.60546875, 0.48874977521574114)

In [96]:
# compile a lookup table of each team - unique team ID and team name...

def table():
    '''
    This defines the total unique teams and their idx numbers...
    '''
    # total teams
    teams = pd.concat([dat.ix[:,3],dat.ix[:,4]])
    teams= teams.reset_index(drop=True)
    teams = pd.DataFrame(pd.unique(teams))
    print(teams)

In [97]:
table()

                      0
0                sylloM
1          Epsilon Hype
2             Fragsport
3       ALTERNATE aTTaX
4                  LDLC
5              NiceShot
6                  ENCE
7               Epsilon
8                fnatic
9                    SK
10          Preparation
11                   G2
12         dream[S]cape
13        Natus Vincere
14              B.O.O.T
15             dignitas
16          AVANT GARDE
17                  NiP
18            Athletico
19                 New4
20          HellRaisers
21          VG.CyberZen
22              onestop
23               Exile5
24           Virtus.pro
25                TyLoo
26            iGame.com
27             FlipSid3
28                 k1ck
29            Immortals
...                 ...
1331     kRoGhs KanonEr
1332           MIXERINO
1333            madjicK
1334   Eternal Conflict
1335       The Flying V
1336          Deponeret
1337            Vikings
1338            DEMGUNZ
1339   kassad's Legends
1340            